In [1]:
import os
import glob
import numpy as np
import cv2
import shutil
import albumentations as A
import os
import random
import splitfolders
import csv
import matplotlib.pyplot as plt

In [7]:
#paths

hub_img = './data/aihub/images/sampled/'
hub_txt = './data/aihub/labels/sampled/'
ss_img = './data/ss/ss_images/sampled/'
ss_txt = './data/ss/ss_labels/sampled/'
taco_bg = './data/taco/images/background/'

img_to = './data/images/'
txt_to = './data/labels/'

In [5]:
img_list = os.listdir(img_to)

In [36]:
eq_hist_sample = random.sample(img_list,6500)

In [47]:
#equalhist

def eq_hist (img_data) :
    img = cv2.cvtColor(img_data, cv2.COLOR_BGR2HSV)
    img[:,:,2] = cv2.equalizeHist(img[:,:,2])
    img = cv2.cvtColor(img,cv2.COLOR_HSV2BGR)
    return img


In [ ]:
for img in eq_hist_sample :
    img_name = img.split('/')[-1].replace('.jpg','')
    new_name = img_name+'_eq'
    img = cv2.imread(img)
    eqed_img = eq_hist(img)
    
    #save img
    cv2.imwrite('./data/images/equal/'+new_name+'.jpg',eqed_img)
    
    #copy txt
    shutil.copy('./data/labels/'+img_name+'.txt','./data/labels/equal/'+new_name+'.txt')

In [48]:
#colorjitter

color_transform = A.Compose([
    A.ColorJitter(brightness=0.3,contrast=0.3,saturation=0.1,hue=0.1,p=1),
])

In [50]:
aug_sample_list = random.sample(img_list,5000)

In [53]:
for img in aug_sample_list :
    img_name = img.split('/')[-1].replace('.jpg','')
    new_name = img_name+'_aug'
    img = cv2.imread(img)
    transformed = color_transform(image=img)
    aug_img = transformed['image']
    cv2.imwrite('./data/images/aug/'+new_name+'.jpg',aug_img)
    
    shutil.copy('./data/labels/'+img_name+'.txt','./data/labels/aug/'+new_name+'.txt')

In [ ]:
#background augmentation

bg_transform = A.Compose([
    A.RandomCrop(width=640,height=640),
    A.ColorJitter(brightness=0.2,contrast=0.2,saturation=0.2,hue=0.2,p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
])

#background image sampling

taco_path = './data/taco/'
taco_img_to = taco_path+'images/'
taco_label_to = taco_path+'labels/'

meta_df = pd.read_csv(taco_path+'meta_df.csv')

img_count = len(os.listdir(taco_img_to+'/background/'))

while img_count < 2000 :
    img_count = len(os.listdir(taco_img_to+'/background/'))
    ran_num = random.randint(0,1499)
    target_datas = meta_df[meta_df['img_id']==ran_num]
    img_w = target_datas.iloc[0][1]
    if img_w < 640 :
        continue
    img_path = target_datas.iloc[0][3]
    img_full_path = taco_path+img_path
    img = cv2.imread(img_full_path)

    #bbox value -> 0
    for j in range(len(target_datas)) :
        data = target_datas.iloc[j]
        x1 =  int(data[8])
        x2 = x1 + int(data[10])
        y1 =  int(data[9])
        y2 = y1 + int(data[11])
        img[y1:y2,x1:x2] = 0

    #apply randomcrop, colorjitter
    transformed = bg_transform(image=img)
    tf_img = transformed['image']

    flat = list(tf_img.flatten())
    if flat.count(0) > len(flat)/5  :
        continue
        
    #save bg img
    cv2.imwrite(taco_img_to+'/background/'+str(img_count+1)+'.jpg',tf_img)
        

In [1]:
#create random overlapped image

In [2]:
#crop object

In [11]:
label_from = './data/labels/train/'
img_from = './data/images/train/'
bg_from= './data/bg_images/train/'
obj_from = './data/images/object/'

img_to = './data/images/overlap/'

In [12]:
txt_list = glob.glob('./data/labels/train/*.txt')

In [8]:
num2class = {
    0: 'furniture',
    1: 'metal',
    2: 'wood',
    3: 'pottery',
    4: 'vinyl',
    5: 'styrofoam',
    6: 'glass',
    7: 'cloth',
    8: 'bike',
    9: 'electro',
    10: 'paper',
    11: 'can',
    12: 'pet',
    13: 'plastic',
    14: 'light',
}

In [26]:
def crop_object(img) :

    w = img.shape[1]
    h = img.shape[0]

    with open(target,'r') as f :
        lines = f.readlines()

        for line in lines :

            line_split = line.split(' ')
            line_split = [i.replace('\n','') for i in line_split]

            class_num = int(line_split[0])
            class_name = num2class[class_num]
            
            xc = float(line_split[1])*w
            yc = float(line_split[2])*h
            bw = float(line_split[3])*w
            bh = float(line_split[4])*h

            x1 = int(xc-bw/2)
            x2 = int(xc+bw/2)
            y1 = int(yc-bh/2)
            y2 = int(yc+bh/2)

    cropped= img[y1:y2,x1:x2]
    
    return cropped, int(bw), int(bh), class_num, class_name


In [27]:
for idx, target in enumerate(txt_list) :
    target_name_txt = target.split('/')[-1]
    target_name_jpg = target_name_txt.replace('.txt','.jpg')
    img = cv2.imread(img_from+target_name_jpg)
    cropped, bw, bh, class_num, class_name = crop_object(img)
    cv2.imwrite(img_to+str(class_num)+'_'+class_name+'_'+str(bw)+'_'+str(bh)+'_'+str(idx)+'.jpg',cropped)

In [ ]:
#load random bg

def pick_bg (bg_list) :
    bg = random.choice(bg_list)
    bg_img = cv2.imread(bg)
    return bg_img

In [3]:
#random load obj

def pick_obj (obj_list) :
    obj = random.choice(obj_list)
    class_num = obj.split('/')[-1].split('_')[0]
    obj_img = cv2.imread(obj)
    return obj_img, class_num

In [4]:
#aug transform

def transform_img(img,max_size) :
    transform = A.Compose([
        A.RandomRotate90(),
        A.RandomScale(scale_limit=(-0.3,0),interpolation=cv2.INTER_AREA)
    ])
    
    tf_img = transform(image=img)
    tf_img = tf_img['image']
    
    transform_max = A.Compose([
        A.LongestMaxSize(max_size=max_size-1)
    ])
    
    if max(tf_img.shape) >= 640 :
        tf_img = transform_max(image=tf_img)
        tf_img = tf_img['image']
        
    
    return tf_img


In [5]:
# create yolo txt

def create_txt(path,name,yolo_format_list) :
    with open(path+name,'w',newline='') as fw :
        write = csv.writer(fw,delimiter=' ')
        write.writerows(yolo_format_list)

In [21]:
def create_aug(bg_from,obj_from,target_num,obj_count,try_num) :
    bg_list = glob.glob(bg_from+'*.jpg')
    bg_list = [i.replace('\\','/') for i in bg_list]
    
    obj_list = glob.glob(obj_from+'*.jpg')
    obj_list = [i.replace('\\','/') for i in obj_list]
    
    k=0
    
    while k < target_num :
        k+=1
    
        # bg load
        bg_img = pick_bg(bg_list)
        bg_w = bg_img.shape[1]
        bg_h = bg_img.shape[0]


        #create list for yolo box label
        yolo_format_list=[]

        i= 0
        
        # maximum try = try_num  ///// maximum obj nums in 1 image = obj_count
        while i<try_num and len(yolo_format_list) < obj_count :
             
            obj_img,class_num = pick_obj(obj_list)
            new_obj_img = transform_img(obj_img,640)

            new_w = new_obj_img.shape[1]
            new_h = new_obj_img.shape[0]

            #new obj (x1,y1), (x2,y2) on bg_img
            obj_x1 = random.randint(0,bg_w-new_w)
            obj_y1 = random.randint(0,bg_h-new_h)
            obj_x2 = obj_x1 + new_w
            obj_y2 = obj_y1 + new_h

            xcenter = ((obj_x1 + obj_x2) /2)/bg_w
            ycenter = ((obj_y1 + obj_y2) /2)/bg_h
            boxw = new_w/bg_w
            boxh = new_h/bg_h
            
            is_cover = False

            for box in yolo_format_list :
                old_xc = box[1]
                old_yc = box[2]
                old_boxw = box[3]
                old_boxh = box[4]
                
                # new object distance from previous objects
                if abs(old_xc - xcenter) < boxw/2 and abs(old_yc - ycenter) < boxh/2 :
                    is_overlap = True
                    break
                    
            # skip if new obj cover previous images
            if is_cover :
                continue

            # put obj on bg_img
            
            bg_img[obj_y1:obj_y2,obj_x1:obj_x2] = new_obj_img

            yolo_format = [class_num,xcenter,ycenter,boxw,boxh]
            yolo_format_list.append(yolo_format)
            
        cv2.imwrite('./data/images/aug/aug_'+str(k)+'.jpg',bg_img)
        create_txt('./data/labels/aug/','aug_'+str(k)+'.txt',yolo_format_list)
        print(f'{k} saved')
            


In [23]:
create_aug(bg_from,obj_from,200,5,10)

1 saved
2 saved
3 saved
4 saved
5 saved
6 saved
7 saved
8 saved
9 saved
10 saved
11 saved
12 saved
13 saved
14 saved
15 saved
16 saved
17 saved
18 saved
19 saved
20 saved
21 saved
22 saved
23 saved
24 saved
25 saved
26 saved
27 saved
28 saved
29 saved
30 saved
31 saved
32 saved
33 saved
34 saved
35 saved
36 saved
37 saved
38 saved
39 saved
40 saved
41 saved
42 saved
43 saved
44 saved
45 saved
46 saved
47 saved
48 saved
49 saved
50 saved
51 saved
52 saved
53 saved
54 saved
55 saved
56 saved
57 saved
58 saved
59 saved
60 saved
61 saved
62 saved
63 saved
64 saved
65 saved
66 saved
67 saved
68 saved
69 saved
70 saved
71 saved
72 saved
73 saved
74 saved
75 saved
76 saved
77 saved
78 saved
79 saved
80 saved
81 saved
82 saved
83 saved
84 saved
85 saved
86 saved
87 saved
88 saved
89 saved
90 saved
91 saved
92 saved
93 saved
94 saved
95 saved
96 saved
97 saved
98 saved
99 saved
100 saved
101 saved
102 saved
103 saved
104 saved
105 saved
106 saved
107 saved
108 saved
109 saved
110 saved
111 save

In [71]:
img = cv2.imread('./data/overlap/images/custom_00003.jpg')
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

with open('./data/overlap/labels/resized/custom_00003.txt','r') as f:
    lines = f.readlines()

boxes = []
for line in lines :

    line_split = line.split(' ')
    cls = line_split[0]
    xc = float(line_split[1])
    yc = float(line_split[2])
    bw = float(line_split[3])
    bh = float(line_split[4].replace('\n',' '))
    new_line = [xc,yc,bw,bh,cls]
    boxes.append(new_line)


In [ ]:
# custom img aug

In [ ]:
txt_from = './data/labels/train/'
img_from = './data/images/train/'

In [202]:
img_list = os.listdir(img_from)

In [203]:
img_list = [i for i in img_list if '.jpg' in i]

In [ ]:
i=0

while i<2000 :
    i+=1
    img_file = random.choice(img_list)
    txt_name = img_file.replace('.jpg','.txt')
    img = cv2.imread(os.path.join(img_from,img_file))
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

    with open(os.path.join(txt_from,txt_name),'r') as f:
        lines = f.readlines()

    boxes = []
    for line in lines :

        line_split = line.split(' ')
        cls = line_split[0]
        xc = max(0,float(line_split[1]))
        yc = max(0,float(line_split[2]))
        bw = max(0,float(line_split[3]))
        bh = max(0,float(line_split[4].replace('\n',' ')))
        new_line = [xc,yc,bw,bh,cls]
        boxes.append(new_line)


    ts = A.Compose([
        A.RandomSizedBBoxSafeCrop(640,640),

        A.ColorJitter(brightness=0.3,contrast=0.3,saturation=0.1,hue=0.1,p=0.5),
        A.RandomRotate90(p=0.5),
        A.Flip(p=0.5),

    ],bbox_params=A.BboxParams(format='yolo'))


    try:
        tfed = ts(image=img,bboxes=boxes)
    except:
        continue
        
    new_img = tfed['image']
    new_label = tfed['bboxes']
    
    new_img = cv2.cvtColor(new_img,cv2.COLOR_RGB2BGR)
    cv2.imwrite(os.path.join(img_to,'aug_'+str(i)+'_'+img_file),new_img)
    
    
    with open(os.path.join(txt_to,'aug_'+str(i)+'_'+txt_name),'w') as fw :
        for bbox in new_label :

            xc = bbox[0]
            yc = bbox[1]
            bw = bbox[2]
            bh = bbox[3]
            cls = bbox[4]
            
            yolo_format = [cls,str(xc),str(yc),str(bw),str(bh)+'\n']
            yolo_str = ' '.join(yolo_format)
            fw.write(yolo_str)
            